In [71]:
#imports.... Run this each time after restarting the Kernel
#!pip install -e git+https://github.com/ibm-watson-data-lab/pixiedust@david-gateway-branch#egg=pixiedust
#!pip install --user --upgrade pixiedust
#!pip install watson_developer_cloud
import pixiedust
import json
import pandas
import datetime
import watson_developer_cloud
import json
import time
import re
import pyspark
import sys
import pandas as pd
from botocore.client import Config
import ibm_boto3
import types


## Load Credentails

In [72]:
# The code was removed by DSX for sharing.

## create Object storage client

In [73]:
client = ibm_boto3.client(service_name='s3', 
    ibm_api_key_id = credentials['IBM_API_KEY_ID'],
    ibm_auth_endpoint = credentials['IBM_AUTH_ENDPOINT'],
    config = Config(signature_version='oauth'),
    endpoint_url = 'https://s3-api.us-geo.objectstorage.service.networklayer.com')





## initialize watson tone analyzer

In [74]:
from watson_developer_cloud import ToneAnalyzerV3

tone_analyzer = ToneAnalyzerV3(version = '2016-05-19',
                               username = username_TON,
                               password = password_TON)

## Helper functions

### process text
details here

### process transcript

In [77]:

chunk_size=30

def chunk_transcript(transcript, chunk_size):
    transcript = transcript.split(' ')
    return [ transcript[i:i+chunk_size] for i in range(0, len(transcript), chunk_size) ] # chunking data
    

def process_text(text):
    transcript=''
    for sentence in json.loads(text)['results']:
        transcript = transcript + sentence['alternatives'][0]['transcript'] # concatenate sentences
    transcript = chunk_transcript(transcript, chunk_size) # chunk the transcript
    return transcript


def analyze_transcript(file_name):
    transcript = client.get_object(Bucket = credentials['BUCKET'], Key = file_name)['Body']
    transcript = transcript.read().decode("utf-8")
    tone_analysis={}
    for chunk in process_text(transcript):
        chunk = ' '.join(chunk)
        print(chunk)
        tone_analysis[chunk] = tone_analyzer.tone(chunk, content_type='text/plain')
    print(tone_analysis)
    res=client.put_object(Bucket = credentials['BUCKET'], Key= file_name[0].split('_')[0]+'_tone', Body = json.dumps(tone_analysis))


## run it

In [79]:
text_files = ['sample1-addresschange-positive_text.json', 'sample2-address-negative_text.json', 'sample3-shirt-return-weather-chitchat_text.json', 'sample4-angryblender-sportschitchat-recovery_text.json', 'sample5-calibration-toneandcontext_text.json'] # we add audio files to COS pre-conference


analyze_transcript(text_files[0])

good morning can you give me some help I'd like to change my address please my name is Ryan Smith I am from Sacramento California that's right my phone number
is five five five one two one two yes that's me my old address is number one two three oak street my new address is five six seven pine street
yes and the zip is nine zero two one zero yep that's right now the phone number stays the same that's right I would like to keep all the options
of said no other changes the only thing that I want to change is the address yes that's right yep very good yes thank you so much for help it
thanks have a good day bye bye 
{"of said no other changes the only thing that I want to change is the address yes that's right yep very good yes thank you so much for help it": {'document_tone': {'tone_categories': [{'tones': [{'tone_name': 'Anger', 'tone_id': 'anger', 'score': 0.008838}, {'tone_name': 'Disgust', 'tone_id': 'disgust', 'score': 0.003648}, {'tone_name': 'Fear', 'tone_id': 'fear', 'score': 0.0